## Tensoflow MLP source

In [2]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''


'\nA Multilayer Perceptron implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [1]:
import json

import tensorflow as tf

import numpy as np
import random
import pickle 
import matplotlib.pyplot as plt
from collections import Counter

 
import vist
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)


## Create dataset

In [2]:
with open("data/img2features", 'rb') as file:
        img2features = pickle.load(file)
        
vist_images_dir = 'data/val.images'
vist_annotations_dir = 'data'
sis = vist.Story_in_Sequence(vist_images_dir, vist_annotations_dir, ['val'])


topic2albums = {} 

for album in sis.Albums.values():
    album_id = album['id']
    vist_label = album["vist_label"]
    
    if not vist_label in topic2albums: 
        topic2albums[vist_label] = [album_id]
    else: 
        topic2albums[vist_label].append(album_id)
        
        
topic2stories = {}

for topic in topic2albums:
    topic2stories[topic] = []
    
    topic_albums = topic2albums[topic];
    for album_id in topic_albums:
        stories = sis.Albums[album_id]['story_ids']
        topic2stories[topic].extend(stories)
        
    

print("Topics (#albums) (#stories)")
for i,key in enumerate(sorted(topic2albums.keys())):         
    print("%d.%s (%d)(%d)"%(i,key, len(topic2albums[key]), len(topic2stories[key])))


sis.Topics = Topics = {topic : {'story_ids' : topic2stories[topic], 'album_ids' : topic2albums[topic]} for topic in topic2stories}


sis's [val] loaded. It took 0.42 seconds.
Make mapping ...
Mapping for [Albums][Images][Stories][Sents] done.
Topics (#albums) (#stories)
0.4th_of_july (44)(220)
1.MISC (60)(300)
2.amusement_park (103)(515)
3.award_ceremony (16)(80)
4.baby_shower (5)(25)
5.bar_mitzvah (8)(40)
6.bat_mitzvah (2)(10)
7.beach (150)(750)
8.bike_race (29)(145)
9.birthday (87)(435)
10.birthday_party (20)(100)
11.breaking_up (77)(385)
12.building_a_house (98)(490)
13.camping (1)(5)
14.car_accident (26)(130)
15.carnival (57)(285)
16.christmas (20)(100)
17.church (47)(235)
18.cinco_de_mayo (20)(100)
19.cooking_dinner (19)(95)
20.day_parade (23)(115)
21.easter (50)(250)
22.eid_al-adha (2)(10)
23.eid_al-fitr (3)(15)
24.engagement (17)(85)
25.fair (14)(70)


In [23]:
def convert_one_hot(labels_list):
    one_hot = [0] * 5
#     for order in labels_list:
#         one_hot[order] = 1
    data = Counter(labels_list)
    one_hot[data.most_common(1)[0][0]] = 1
    return one_hot

#get images that belong in a story
img2labels = {}
for im_name in img2features:
    img_id = im_name.split('.')[0]
    img2labels[img_id] = []
    
for story in sis.Stories.values():
    for order, img_id in enumerate(story['img_ids']):
        img2labels[img_id].append(order)
    
# shuffle data
# r_img_id = random.choice(list(img2labels.keys()))
# print(r_img_id)
# print(img2labels[r_img_id])

img2labels = {img_id : convert_one_hot(img2labels[img_id]) for img_id in img2labels if img2labels[img_id]}


#keep images that belong in relevant topics
relevant_topics = ["4th_of_july", "bike_race", "award_ceremony", "birthday"]
relevant_img_ids = set([])

for topic in relevant_topics:
    for story_id in sis.Topics[topic]["story_ids"]:
        relevant_img_ids.update(sis.Stories[story_id]["img_ids"])

img2labels = {img_id : img2labels[img_id] for img_id in img2labels if img_id in relevant_img_ids}
        
        
print("Images that belong in relevant topics: %d", %(len(relevant_img_ids)))

 

1408


In [24]:
X = np.zeros((len(img2labels), 4096))
Y = np.zeros((len(img2labels), 5))

for i, img_id in enumerate(img2labels):

    X[i,:] = img2features[img_id + '.jpg']
    Y[i,:] = img2labels[img_id]
    
train_per = 0.8
train_features = X[:int(0.8*X.shape[0]), :]
train_labels = Y[:int(0.8*X.shape[0]), :]

test_features = X[int(0.8*X.shape[0]):, :]
test_labels = Y[int(0.8*X.shape[0]):, :]

print("Train/test split: %d%% / %d %%" % (train_per * 100,(1-train_per) * 100))
print("Train size: %d" %(len(train_features)))
print("Test size: %d" %(len(test_features)))

(1-train_per)

Train/test split: 80% / 19 %
Train size: 1126
Test size: 282


0.19999999999999996

## Classify

In [29]:
def get_batch(X, Y, current_batch, batch_size):
    batch_x = X[current_batch*batch_size : (current_batch+1)*batch_size, :]
    batch_y = Y[current_batch*batch_size : (current_batch+1)*batch_size, :]
    #print(current_batch*batch_size, (current_batch+1) * batch_size)
    return batch_x, batch_y

# Parameters
learning_rate = 0.001
training_epochs = 50
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 512 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 4096 # MNIST data input (img shape: 28*28)
n_classes = 5 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

keep_prob = tf.placeholder(tf.float32)


# Create model
def multilayer_perceptron(x, weights, biases, keep_prob):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
 


# Construct model
pred = multilayer_perceptron(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = get_batch(train_features, train_labels, i, batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y,
                                                          keep_prob: 0.2})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print(" Train accuracy:", accuracy.eval(feed_dict = {x: train_features, y: train_labels, keep_prob : 1.0}))
            
            print(" Test accuracy:", accuracy.eval(feed_dict = {x: test_features, y: test_labels, keep_prob : 1.0}))
    print("Optimization Finished!")

    # Test model
    print("Accuracy:", accuracy.eval(feed_dict ={x: test_features, y: test_labels, keep_prob : 1.0}))


Epoch: 0001 cost= 16225.125710227
 Train accuracy: 0.214032
 Test accuracy: 0.202128
Epoch: 0002 cost= 13396.907049006
 Train accuracy: 0.23357
 Test accuracy: 0.180851
Epoch: 0003 cost= 12050.147460938
 Train accuracy: 0.262877
 Test accuracy: 0.184397
Epoch: 0004 cost= 10702.319602273
 Train accuracy: 0.280639
 Test accuracy: 0.212766
Epoch: 0005 cost= 9729.784446023
 Train accuracy: 0.293961
 Test accuracy: 0.216312
Epoch: 0006 cost= 9271.381081321
 Train accuracy: 0.324156
 Test accuracy: 0.216312
Epoch: 0007 cost= 8561.419611151
 Train accuracy: 0.343695
 Test accuracy: 0.22695
Epoch: 0008 cost= 7786.229980469
 Train accuracy: 0.360568
 Test accuracy: 0.22695
Epoch: 0009 cost= 7843.852627841
 Train accuracy: 0.376554
 Test accuracy: 0.205674
Epoch: 0010 cost= 6719.494273793
 Train accuracy: 0.402309
 Test accuracy: 0.191489
Epoch: 0011 cost= 6497.236550071
 Train accuracy: 0.437833
 Test accuracy: 0.212766
Epoch: 0012 cost= 6546.509499290
 Train accuracy: 0.455595
 Test accuracy: 

In [11]:
sis.Sents['223009']

{'album_id': '72157623139776636',
 'id': '223009',
 'img_id': '4250204464',
 'length': 8,
 'order': 4,
 'original_text': 'I myself prefer to use my feet.',
 'setting': 'first-2-pick-and-tell',
 'story_id': '44601',
 'text': 'i myself prefer to use my feet .',
 'tier': 'story-in-sequence',
 'worker_id': 'AS2MDFCRUS236TU'}